In [1]:
import sys

modules_dir = 'D:/usr14/project/Binary/adiabaticbinary/ww'
sys.path.insert(0, modules_dir)

In [2]:
from modules import resnet1
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from readdata import read_dataset

In [3]:
device=torch.device('cuda')
def fit(model:resnet1.ResNet, optim,lossfunc,trainloader: DataLoader):
    model.train()
    totalloss = 0
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optim.zero_grad()
        output = model(data).to(device)
        loss = lossfunc(output, target)
        loss.backward()
        optim.step()
        with torch.no_grad():
            totalloss += loss.item()*data.size(0)
    return totalloss/len(trainloader.sampler)

def evaluate(model:resnet1.ResNet, val_loader: DataLoader,lossfunc:nn.CrossEntropyLoss):
    model.eval()
    loss=0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).to(device)
            loss+=lossfunc(outputs,labels).item()*inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct+=(predicted==labels).sum().item()

    return loss/len(val_loader.sampler) ,correct / total

In [4]:
model = resnet1.ResNet(False, False).to(device)
lossfunc=nn.CrossEntropyLoss().to(device)
train_loader,val_loader,test_loader=read_dataset(128)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
lr=0.1
counter=0
min_val_loss=np.inf
for epoch in range(150):
    if counter/10==1:
        counter=0
        lr*=0.5
        print(f"lr reduced to {lr}")
    optim=torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    loss=fit(model,optim,lossfunc,train_loader)
    val_loss,val_acc=evaluate(model,val_loader,lossfunc)
    print(f'epoch: {epoch+1}, loss: {loss}, val_loss: {val_loss}, val_acc:{val_acc}')
    if val_loss<min_val_loss:
        min_val_loss=val_loss
        print(f"val loss reduced to {min_val_loss}")
        counter=0
        torch.save(model.state_dict(), 'pre1.pth')
    else:
        counter+=1

epoch: 1, loss: 2.186323071861267, val_loss: 1.7741397729873658, val_acc:0.3373
val loss reduced to 1.7741397729873658
epoch: 2, loss: 1.6957969888687134, val_loss: 1.5476026124954223, val_acc:0.4195
val loss reduced to 1.5476026124954223
epoch: 3, loss: 1.5325230318069458, val_loss: 1.374797057914734, val_acc:0.4951
val loss reduced to 1.374797057914734
epoch: 4, loss: 1.3669288436889648, val_loss: 1.1922884119987487, val_acc:0.5706
val loss reduced to 1.1922884119987487
epoch: 5, loss: 1.2195643501281739, val_loss: 1.5068196214675904, val_acc:0.5185
epoch: 6, loss: 1.1108315605163575, val_loss: 1.0334801809310914, val_acc:0.6392
val loss reduced to 1.0334801809310914
epoch: 7, loss: 1.0070762053489686, val_loss: 1.0077053094863893, val_acc:0.6573
val loss reduced to 1.0077053094863893
epoch: 8, loss: 0.9085407932281494, val_loss: 0.8490832136154175, val_acc:0.7015
val loss reduced to 0.8490832136154175
epoch: 9, loss: 0.8446003293991089, val_loss: 1.1524074582099915, val_acc:0.6232
e